https://python.langchain.com/v0.2/docs/how_to/structured_output/
# How to return structured data from model
This is a concept to matches the output from model with specific schema
## The `.with_structured_output()` method
`.with_structured_output()` is implemented for models that provide native APIs for structuring output like a 

**NOTE:** model are supported [Link](https://python.langchain.com/v0.2/docs/integrations/chat/)

In [ ]:
# https://python.langchain.com/v0.2/docs/how_to/structured_output/#advanced-specifying-the-method-for-structuring-outputs
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

In [ ]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")
# Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!', rating=None)

In [ ]:
# Choosing between multiple schemas
from typing import Union
class ConversationalResponse(BaseModel):
  """Respond in a conversational manner. Be kind and helpful."""
  response: str = Field(description="A conversational response to the user's query")

class Response(BaseModel):
  output: Union[Joke, ConversationalResponse]

structured_llm = llm.with_structured_output(Response)

In [ ]:
structured_llm.invoke("Tell me a joke about cats")
# Response(output=Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!', rating=8))

In [ ]:
structured_llm.invoke("How are you today?")
# Response(output=ConversationalResponse(response="I'm just a digital assistant, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"))

In [ ]:
# Apply with streaming
json_schema = {
  "title": "joke",
  "description": "Joke to tell user.",
  "type": "object",
  "properties": {
    "setup": {
      "type": "string",
      "description": "The setup of the joke",
    },
    "punchline": {
      "type": "string",
      "description": "The punchline to the joke",
    },
    "rating": {
      "type": "integer",
      "description": "How funny the joke is, from 1 to 10",
    },
  },
  "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

for chunk in structured_llm.stream("Tell me a joke about cats"):
  print(chunk)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")